### 2.5 Digit recognition using Tensorflow (thinning)

In [1]:
import cv2
import numpy as np

#1
net = cv2.dnn.readNetFromTensorflow('./dnn/MNIST_MLP_frozen_graph.pb')
#net = cv2.dnn.readNetFromTensorflow('./dnn/MINIST_CNN_frozen_graph2.pb')

#2
def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(dst, (x, y), 15, (255, 255, 255), -1)
    cv2.imshow('dst', dst)

def thinning(A):
    skel_dst = np.zeros(A.shape, np.uint8)
    B = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize = (3, 3))
    done = True
    while done:
        erode = cv2.erode(A, B)
        opening = cv2.morphologyEx(erode, cv2.MORPH_OPEN, B)
        tmp = cv2.subtract(erode, opening)    # cv2.absdiff(erode, opening)
        skel_dst = cv2.bitwise_or(skel_dst, tmp)
        A = erode.copy()
        done = cv2.countNonZero(A) != 0
    skel_dst = cv2.dilate(skel_dst, None, 4)    
    skel_dst = cv2.erode(skel_dst, None, 5)
    return skel_dst
           
dst  = np.zeros(shape=(512, 512, 3), dtype=np.uint8)
cv2.namedWindow('dst')
cv2.setMouseCallback('dst', onMouse)


mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
font = cv2.FONT_HERSHEY_SIMPLEX
x_img = np.zeros(shape=(28, 28), dtype=np.uint8)

#3
while True:
    key = cv2.waitKey(25)    
    if key == 27: 
        break;
    elif key == ord('r'):
        dst[:,:] = 0
        cv2.imshow('dst',dst)
    elif key == ord(' '):
        gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
        _, contours, _ = cv2.findContours(gray, mode, method)

        for i, cnt in enumerate(contours):
#3-1
            x, y, width, height = cv2.boundingRect(cnt)
            cv2.rectangle(dst, (x, y), (x+width, y+height), (0,0,255), 2)
            cx, cy = x + width/2, y + height/2
            if width > height:
                r = width/2
            else:
                r = height/2            
            cx, cy, r= int(cx), int(cy), int(r)
            img = gray[cy-r:cy+r, cx-r:cx+r]
            img = thinning(img)
            cv2.imshow('thinning', img)
            img = cv2.resize(img, dsize=(20, 20),interpolation=cv2.INTER_AREA)
            ret, img = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)
            x_img[:,:] = 0
            x_img[4:24, 4:24] = img
            x_img = cv2.dilate(x_img, None, 2)
            x_img = cv2.erode(x_img, None, 4)
            x_img1 = thinning(x_img)
            cv2.imshow('x_img', x_img)
#3-2
            blob = cv2.dnn.blobFromImage(x_img) # blob.shape=(1, 1, 28, 28)
            print('blob.shape=', blob.shape)

            net.setInput(blob)
            res = net.forward()
            y_predict = np.argmax(res, axis = 1)
            print('y_predict=', y_predict)
            digit = int(y_predict[0])
            cv2.putText(dst, str(digit), (x, y), font, 3, (255,0,0), 5)
        
        cv2.imshow('dst',dst)
cv2.destroyAllWindows()

blob.shape= (1, 1, 28, 28)
y_predict= [3]
blob.shape= (1, 1, 28, 28)
y_predict= [2]
blob.shape= (1, 1, 28, 28)
y_predict= [3]
blob.shape= (1, 1, 28, 28)
y_predict= [7]
blob.shape= (1, 1, 28, 28)
y_predict= [5]


## 3. Object Detection

### 3.1 Face and Eye Detection

In [2]:
# ref: https://docs.opencv.org/3.4.1/d7/d8b/tutorial_py_face_detection.html
import numpy as np
import cv2

faceCascade= cv2.CascadeClassifier(
      './haarcascades/haarcascade_frontalface_default.xml')
eyeCascade = cv2.CascadeClassifier(
    './haarcascades/haarcascade_eye.xml')

src = cv2.imread('./data/lena.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray, 1.1, 3) #(gray, 1.1, 0)

for (x, y, w, h) in faces:
    cv2.rectangle(src, (x,y),(x+w, y+h),(255,0,0), 2)
    
    roi_gray  = gray[y:y+h, x:x+w]
    roi_color = src[y:y+h, x:x+w]
    
    eyes = eyeCascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        
cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

### 3.2 Face Detection on YouTube

In [3]:
'''
 pip install youtube_dl
 pip install pafy
'''
import numpy as np
import cv2, pafy

faceCascade= cv2.CascadeClassifier(
      './haarcascades/haarcascade_frontalface_default.xml')

url = 'https://www.youtube.com/watch?v=S_0ikqqccJs'
video = pafy.new(url)
print('title = ', video.title)

best = video.getbest(preftype='webm')
print('best.resolution', best.resolution)

file = best.download(filepath="./data/demovideo2." + best.extension)
cap = cv2.VideoCapture(file)
#cap=cv2.VideoCapture(best.url)  ### I want to make an url object but failed

while(True):
        retval, frame = cap.read()
        if not retval:
                break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = faceCascade.detectMultiScale(gray) #minSize=(50, 50)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x,y),(x+w, y+h),(255,0,0), 2)           
        cv2.imshow('frame',frame)
 
        key = cv2.waitKey(25)
        if key == 27: # Esc
                break
                
cv2.destroyAllWindows()

title =  참이슬 아이유 X 박서준 바이럴영상(30")
best.resolution 640x360


## 4. Face Recognition

### 4.1 face recognition with AT&T Face Database 1: EigenFaceRecognizer, FisherFaceRecognizer

In [4]:
import cv2
import numpy as np
import random

#1
WIDTH = 92
HEIGHT = 112
def load_face(filename='./data/faces.csv', test_ratio=0.2):
    file = open(filename, 'r')
    lines = file.readlines()
    
    N = len(lines)
    faces = np.empty((N, WIDTH*HEIGHT), dtype=np.uint8 )
    labels = np.empty(N, dtype = np.int32)
    
    for i, line in enumerate(lines):
        filename, label = line.strip().split(';')
        labels[i] = int(label)
        img = cv2.imread(filename[:2]+'data/'+filename[2:], cv2.IMREAD_GRAYSCALE) # unzip ./data/att_faces.zip
        faces[i, :] = img.flatten()

    # shuffling and seperate train and test data
    indices = list(range(N))
    random.seed(1) # same random sequences, so the same result
    random.shuffle(indices)
    shuffle_faces = faces[indices]
    shuffle_labels = labels[indices]

    test_size = int(test_ratio*N)

    test_faces = shuffle_faces[:test_size]
    test_labels = shuffle_labels[:test_size]

    train_faces = shuffle_faces[test_size:]
    train_labels = shuffle_labels[test_size:]
    return train_faces, train_labels, test_faces, test_labels

#2
train_faces, train_labels, test_faces, test_labels = load_face()
print('train_faces.shape=',  train_faces.shape)
print('train_labels.shape=', train_labels.shape)
print('test_faces.shape=',   test_faces.shape)
print('test_labels.shape=',  test_labels.shape)

#3    
recognizer = cv2.face.EigenFaceRecognizer_create()
##recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.train(train_faces.reshape(-1, HEIGHT, WIDTH), train_labels)

#4: display eigen Face
eigenFace = recognizer.getEigenVectors()
eigenFace = eigenFace.T
print('eigenFace.shape=',  eigenFace.shape)

dst = np.zeros((8*HEIGHT, 10*WIDTH), dtype=np.uint8)

##for i in range(39): # FisherFaceRecognizer
for i in range(80):
    x = i%10
    y = i//10
    x1 = x*WIDTH
    y1 = y*HEIGHT
    x2 = x1+WIDTH
    y2 = y1+HEIGHT  
  
    img = eigenFace[i].reshape(HEIGHT, WIDTH)
    dst[y1:y2, x1:x2] = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)

cv2.imshow('eigenFace 80', dst)

#5: predict test_faces using recognizer
correct_count = 0
for i, face in enumerate(test_faces): 
    predict_label, confidence = recognizer.predict(face)
    if test_labels[i]== predict_label:
        correct_count+= 1
    print('test_labels={}: predicted:{}, confidence={}'.format(
                     test_labels[i], predict_label, confidence))
    
accuracy = correct_count / float(len(test_faces))
print('accuracy=', accuracy)

cv2.waitKey()
cv2.destroyAllWindows()

train_faces.shape= (320, 10304)
train_labels.shape= (320,)
test_faces.shape= (80, 10304)
test_labels.shape= (80,)
eigenFace.shape= (320, 10304)
test_labels=20: predicted:20, confidence=2015.8359348595566
test_labels=15: predicted:15, confidence=1121.1869835189796
test_labels=26: predicted:26, confidence=1275.1992785732932
test_labels=33: predicted:33, confidence=3076.704885172979
test_labels=38: predicted:38, confidence=2466.7371983988214
test_labels=21: predicted:21, confidence=2678.4209967532556
test_labels=32: predicted:32, confidence=2515.5599295133247
test_labels=34: predicted:34, confidence=2074.356117871548
test_labels=0: predicted:0, confidence=3474.3619948362534
test_labels=28: predicted:28, confidence=2123.33104098201
test_labels=5: predicted:5, confidence=2562.925080313562
test_labels=4: predicted:4, confidence=1333.2051538304086
test_labels=3: predicted:3, confidence=2273.283420728049
test_labels=28: predicted:28, confidence=2909.4165933806676
test_labels=22: predicted:22, 

### 4.2 face recognition with AT&T Face Database 2: LBPHFaceRecognizer

In [5]:
import cv2
import numpy as np
import random

#1
WIDTH = 92
HEIGHT = 112
def load_face(filename='./data/faces.csv', test_ratio=0.2):
    file = open(filename, 'r')
    lines = file.readlines()

    N = len(lines)
    faces = np.empty((N, WIDTH*HEIGHT), dtype=np.uint8 )
    labels = np.empty(N, dtype = np.int32)
    for i, line in enumerate(lines):
        filename, label = line.strip().split(';')
        labels[i] = int(label)
        img = cv2.imread(filename[:2]+'data/'+filename[2:], cv2.IMREAD_GRAYSCALE) # unzip ./data/att_faces.zip
        faces[i, :] = img.flatten()

    # shuffling and seperate train and test data
    indices = list(range(N))
    random.seed(1) # same random sequences, so the same result
    random.shuffle(indices)
    shuffle_faces = faces[indices]
    shuffle_labels = labels[indices]

    test_size = int(test_ratio*N)

    test_faces = shuffle_faces[:test_size]
    test_labels = shuffle_labels[:test_size]

    train_faces = shuffle_faces[test_size:]
    train_labels = shuffle_labels[test_size:]
    return train_faces, train_labels, test_faces, test_labels

train_faces, train_labels, test_faces, test_labels = load_face()
print('train_faces.shape=',  train_faces.shape)
print('train_labels.shape=', train_labels.shape)
print('test_faces.shape=',   test_faces.shape)
print('test_labels.shape=',  test_labels.shape)

#2    
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(train_faces.reshape(-1, HEIGHT, WIDTH), train_labels)
 
#3: predict test_faces using recognizer
correct_count = 0
for i, face in enumerate(test_faces.reshape(-1, HEIGHT, WIDTH)):    
    predict_label, confidence = recognizer.predict(face)
    if test_labels[i]== predict_label:
        correct_count+= 1
    print('test_labels={}: predicted:{}, confidence={}'.format(
                     test_labels[i], predict_label,confidence))
    
accuracy = correct_count / float(len(test_faces))
print('accuracy=', accuracy)

train_faces.shape= (320, 10304)
train_labels.shape= (320,)
test_faces.shape= (80, 10304)
test_labels.shape= (80,)
test_labels=20: predicted:20, confidence=63.53159239674068
test_labels=15: predicted:15, confidence=56.76836690095442
test_labels=26: predicted:26, confidence=65.35428328380661
test_labels=33: predicted:33, confidence=78.94946641055041
test_labels=38: predicted:38, confidence=75.97988206043449
test_labels=21: predicted:21, confidence=82.08887829286003
test_labels=32: predicted:32, confidence=84.18183984479495
test_labels=34: predicted:34, confidence=72.16901863930305
test_labels=0: predicted:0, confidence=78.75554190227402
test_labels=28: predicted:28, confidence=63.16955227973658
test_labels=5: predicted:5, confidence=69.28946916637393
test_labels=4: predicted:4, confidence=64.63933640341054
test_labels=3: predicted:3, confidence=66.44944519613094
test_labels=28: predicted:28, confidence=73.32473172587548
test_labels=22: predicted:22, confidence=61.721612437042324
test_lab

### 4.3 face recognition with AT&T Face Database 3: build EigenFace without Recognizer

In [6]:
import cv2
import numpy as np
import random

#1
WIDTH = 92
HEIGHT = 112
def load_face(filename='./data/faces.csv', test_ratio=0.2):
    file = open(filename, 'r')
    lines = file.readlines()

    N = len(lines)
    faces = np.empty((N, WIDTH*HEIGHT), dtype=np.uint8 )
    labels = np.empty(N, dtype = np.int32)
    for i, line in enumerate(lines):
        filename, label = line.strip().split(';')
        labels[i] = int(label)
        img = cv2.imread(filename[:2]+'data/'+filename[2:], cv2.IMREAD_GRAYSCALE) # unzip ./data/att_faces.zip
        faces[i, :] = img.flatten()

    # shuffling and seperate train and test data
    indices = list(range(N))
    random.seed(1) # same random sequences, so the same result
    random.shuffle(indices)
    shuffle_faces = faces[indices]
    shuffle_labels = labels[indices]

    test_size = int(test_ratio*N)

    test_faces = shuffle_faces[:test_size]
    test_labels = shuffle_labels[:test_size]

    train_faces = shuffle_faces[test_size:]
    train_labels = shuffle_labels[test_size:]
    return train_faces, train_labels, test_faces, test_labels

train_faces, train_labels, test_faces, test_labels = load_face()
print('train_faces.shape=',  train_faces.shape)
print('train_labels.shape=', train_labels.shape)
print('test_faces.shape=',   test_faces.shape)
print('test_labels.shape=',  test_labels.shape)

#2
N = 80
mean, eigenFace = cv2.PCACompute(train_faces, mean=None, maxComponents=N)
print('mean.shape = ', mean.shape)
print('eigenFace.shape=',  eigenFace.shape)

Y = cv2.PCAProject(train_faces, mean, eigenFace) # train result
print('Y.shape=', Y.shape)

#3: display eigen Face
dst = np.zeros((8*HEIGHT, 10*WIDTH), dtype=np.uint8)
for i in range(N):
    x = i%10
    y = i//10
    x1 = x*WIDTH
    y1 = y*HEIGHT
    x2 = x1+WIDTH
    y2 = y1+HEIGHT  
  
    img = eigenFace[i].reshape(HEIGHT, WIDTH)
    dst[y1:y2, x1:x2] = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)

cv2.imshow('eigenFace %s'%N, dst)

#4: predict test_faces using  y = eigenFace(face - mean) and min distance
correct_count = 0
for i, face in enumerate(test_faces): 
    y =cv2.PCAProject(face.reshape(1,-1), mean, eigenFace)
    dist=np.sqrt(np.sum((Y-y)**2,axis=1))
    k = np.argmin(dist)
    predict_label = train_labels[k]

    if test_labels[i]== predict_label:
        correct_count+= 1
    print('test_labels={}: predicted:{}'.format(
                     test_labels[i], predict_label))
    
accuracy = correct_count / float(len(test_faces))
print('accuracy=', accuracy)

cv2.waitKey()
cv2.destroyAllWindows()

train_faces.shape= (320, 10304)
train_labels.shape= (320,)
test_faces.shape= (80, 10304)
test_labels.shape= (80,)
mean.shape =  (1, 10304)
eigenFace.shape= (80, 10304)
Y.shape= (320, 80)
test_labels=20: predicted:20
test_labels=15: predicted:15
test_labels=26: predicted:26
test_labels=33: predicted:33
test_labels=38: predicted:38
test_labels=21: predicted:21
test_labels=32: predicted:32
test_labels=34: predicted:34
test_labels=0: predicted:0
test_labels=28: predicted:28
test_labels=5: predicted:5
test_labels=4: predicted:4
test_labels=3: predicted:3
test_labels=28: predicted:28
test_labels=22: predicted:22
test_labels=19: predicted:19
test_labels=23: predicted:23
test_labels=30: predicted:30
test_labels=10: predicted:10
test_labels=21: predicted:21
test_labels=6: predicted:6
test_labels=32: predicted:32
test_labels=38: predicted:38
test_labels=2: predicted:2
test_labels=9: predicted:9
test_labels=16: predicted:16
test_labels=31: predicted:31
test_labels=38: predicted:38
test_labels=35: